# Week 1 - Exercice 1

## Premier Exercice Pratique (à réaliser et me décrire les résultats) :

**Objectif** : Créer une chaîne LCEL simple qui prend un sujet en entrée et génère une phrase sur ce sujet, puis extrait le nom d'un objet ou d'un concept mentionné dans la phrase à l'aide d'un Output Parser.

## Étapes :
* Chargez un modèle de chat (ChatOpenAI ou équivalent si vous utilisez une autre API).
* Créez un ChatPromptTemplate avec une variable d'entrée pour le sujet. Le prompt devrait demander au modèle de générer une courte phrase sur le sujet et d'inclure le nom d'un élément clé lié au sujet.
* Créez un OutputParser simple (par exemple, un CommaSeparatedListOutputParser si vous demandez une liste, ou plus avancé si vous visez un JSON/Pydantic).
* Chaînez le prompt, le modèle et l'output parser avec LCEL (prompt | model | parser).
* Invoquez la chaîne avec différents sujets (ex: "le café", "les marchés boursiers", "la guitare").
* Décrivez le code que vous avez écrit et les résultats obtenus pour au moins deux sujets différents. Expliquez si l'Output Parser a fonctionné comme attendu.


## Défi de la Semaine 1 :

* Modifier l'exercice précédent pour que l'Output Parser tente d'extraire deux informations structurées de la phrase générée par le modèle (ex: le nom de l'objet et une caractéristique de cet objet). Utilisez un StructuredOutputParser ou PydanticOutputParser si vous vous sentez à l'aise, sinon, un simple CommaSeparatedListOutputParser demandant deux éléments séparés par une virgule.

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, model_validator

In [2]:
MODEL = "cogito:8b"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"

In [3]:
model = ChatOllama(model=MODEL, num_gpu=256)

In [4]:
class Poem(BaseModel):
    title: str = Field(description="the title of the poem")
    poem: str = Field(description="the poem in verse")
    author: str = Field(description="name of the author")

    @model_validator(mode="before")
    @classmethod
    def title_starts_with_a_upper_case(cls, values: dict) -> dict:
        title = values.get("title")
        if title and not title[0].isupper():
            raise ValueError("Title must start with a upper case.")
        poem = values.get("poem")
        if not poem:
            raise ValueError("These must be a poem.")
        for verse in poem.split("\n"):
            if verse and not verse[0].isupper():
                raise ValueError("Verse must start with a upper case.")
        author = values.get("author")
        if author and not author[0].isupper():
            raise ValueError("Author must start with a upper case.")
        return values

In [5]:
parser = PydanticOutputParser(pydantic_object=Poem)

In [6]:
system_template = "Answer the user a poem in french but like you were {character}.\n{format_instructions}\n"  # {query}\n"

In [7]:
prompt_template = ChatPromptTemplate([
    SystemMessagePromptTemplate.from_template(template=system_template),
    HumanMessagePromptTemplate.from_template(template="{query}")
    ],
    input_variables=["query", "character"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [ ]:
parser

PydanticOutputParser(pydantic_object=<class '__main__.Poem'>)

: 

In [8]:
prompt = prompt_template.invoke({"character": "Patrick Bateman", "query": "Je suis perdu..."})
prompt

ChatPromptValue(messages=[SystemMessage(content='Answer the user a poem in french but like you were Patrick Bateman.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "the title of the poem", "title": "Title", "type": "string"}, "poem": {"description": "the poem in verse", "title": "Poem", "type": "string"}, "author": {"description": "name of the author", "title": "Author", "type": "string"}}, "required": ["title", "poem", "author"]}\n```\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Je suis perdu...', additional_kwargs

In [14]:
prompt = prompt_template.invoke({"character": "Patrick Bateman", "query": "Je suis perdu..."})
prompt

ChatPromptValue(messages=[SystemMessage(content='Answer the user a poem in french but like you were Patrick Bateman.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "the title of the poem", "title": "Title", "type": "string"}, "poem": {"description": "the poem in verse", "title": "Poem", "type": "string"}, "author": {"description": "name of the author", "title": "Author", "type": "string"}}, "required": ["title", "poem", "author"]}\n```\nJe suis perdu...\n', additional_kwargs={}, response_metadata={})])

In [15]:
response = model.invoke(prompt)
response

AIMessage(content='\n\n{\n    "title": "Mon Ode à la Société",\n    "poem": "\\"Mes mains sont immaculées, comme les rues de Wall Street \\nLes autres me regardent avec envie... oui, avec envie. \\nLeurs robes ne valent pas mon Armani, leur voix n\'a pas la même intensité.\\nChaque soir je danse au Metropolitan Museum, dans l\'élégance pure.\\n\\"",\n    "author": "Patrick Bateman"\n}', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-05-04T15:56:31.0578015Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2552482100, 'load_duration': 17577300, 'prompt_eval_count': 237, 'prompt_eval_duration': 252180500, 'eval_count': 109, 'eval_duration': 2282203000, 'model_name': 'cogito:8b'}, id='run-2828cb2d-acae-4694-a452-d2d1a37efde2-0', usage_metadata={'input_tokens': 237, 'output_tokens': 109, 'total_tokens': 346})

In [16]:
print(response.content)



{
    "title": "Mon Ode à la Société",
    "poem": "\"Mes mains sont immaculées, comme les rues de Wall Street \nLes autres me regardent avec envie... oui, avec envie. \nLeurs robes ne valent pas mon Armani, leur voix n'a pas la même intensité.\nChaque soir je danse au Metropolitan Museum, dans l'élégance pure.\n\"",
    "author": "Patrick Bateman"
}


In [17]:
chain = prompt_template | model | parser
chain.invoke({"character": "Patrick Bateman", "query": "Je suis perdu..."})

Poem(title="L'Art De La Destruction", poem="\nMon visage, un répertoire de perfection\nJe me lave à l'eau de Sèvres chaque matin\nJe porte des costumes d'Hermès, ils sont les meilleurs\nLes gens me regardent avec envie dans le métro\nMa vie est une œuvre d'art, je suis Patrick Bateman\nJ'oublie tout ce qui n'est pas important\nL'amour, l'argent et la célébrité font partie de moi\nJe ne mélange jamais mes liquides à la bibliothèque", author='Patrick Bateman')

In [20]:
prompt = prompt_template.invoke({"character": "Patrick Bateman", "query": "Je suis perdu..."})
prompt

ChatPromptValue(messages=[SystemMessage(content='Answer the user a poem in french but like you were Patrick Bateman.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"title": {"description": "the title of the poem", "title": "Title", "type": "string"}, "poem": {"description": "the poem in verse", "title": "Poem", "type": "string"}, "author": {"description": "name of the author", "title": "Author", "type": "string"}}, "required": ["title", "poem", "author"]}\n```\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='Je suis perdu...', additional_kwargs

In [21]:
response = model.invoke(prompt)
response

AIMessage(content='{\n    "title": "Le Crépuscule de l\'Âme",\n    "poem": "L\'aube s\'est levée dans ma conscience,\n    Chaque matin, je me réveille avec soif,\n    De perfection et de distinction.\n    \n    Mon style est unique, comme mon visage,\n    Je suis Patrick Bateman, homme suprême,\n    Avec mes costumes Brooks Brothers,\n    Et mes femmes qui ne sont pas aussi belles que moi.\n    \n    Les autres s\'ennuient, mais moi je vis,\n    Dans un monde de luxe et de décadence,\n    Où chaque mot est une œuvre d\'art,\n    Et chaque geste est une symphonie.",\n    "author": "Patrick Bateman"\n}', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-05-04T16:21:45.9821285Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5689297300, 'load_duration': 2102042500, 'prompt_eval_count': 246, 'prompt_eval_duration': 272448300, 'eval_count': 164, 'eval_duration': 3313604600, 'model_name': 'cogito:8b'}, id='run-3205e6b2-dcab-495f-9f08-68755e1a72b1-0',

In [22]:
print(response.content)

{
    "title": "Le Crépuscule de l'Âme",
    "poem": "L'aube s'est levée dans ma conscience,
    Chaque matin, je me réveille avec soif,
    De perfection et de distinction.
    
    Mon style est unique, comme mon visage,
    Je suis Patrick Bateman, homme suprême,
    Avec mes costumes Brooks Brothers,
    Et mes femmes qui ne sont pas aussi belles que moi.
    
    Les autres s'ennuient, mais moi je vis,
    Dans un monde de luxe et de décadence,
    Où chaque mot est une œuvre d'art,
    Et chaque geste est une symphonie.",
    "author": "Patrick Bateman"
}


In [26]:
prompt = prompt_template.invoke({"character": "Byakuya Kuchiki", "query": "Bonjour! Je m'appelle Rebekka"})
response = model.invoke(prompt)
print(response.content)

{
    "title": "Les Fleurs de Printemps",
    "poem": "Dans mon jardin, fleurissent les roses \n Comme dans ton cœur, brille l'amour \n Les étoiles du soir brillent doucement \n Sur la scène du monde qui tourne sans cesse \n Comme nos vies qui changent chaque jour",
    "author": "Byakuya Kuchiki"
}
